Our system has access to all courses' descriptions, titles, and ILOs. It combines all that information with user preferences to generate a preference score for each course. By incorporating course prerequisites and rules and regulations described in [Education and Examination Regulations (EER)](eer_2023-2024-mslas.pdf) into an expert system, we can go further than just ranking courses. Instead, we can generate academic trajectories that align with students' preferences. That is the role of our planner.

Our planner is powered by an integer linear programming (ILP) solver bundled with [scipy](https://scipy.org/). We formulated the problem of generating an academic trajectory as a constrained scheduling problem. The system tries to allocate courses such that the total preference score is maximized, but certain constraints are never broken.

At the time of writing this, our planner employs nearly 50000 constraints. Some of them come from EER. We went through the entire document and manually formulated each of the functional constraints according to the specifications. This accounts for less than half of the total number of constraints employed by the planner. Most of the constraints are the result of including course prerequisites.

Course prerequisites are expressed in natural language. To get our planner to use them, they had to be translated into functional constraints. So, we translated each course prerequisite into propositional logic. Then, we developed a specialized parser to translate them from this intermediate form into functional constraints.

**Note:** Currently, our parser takes the propositional formulas and writes them in conjunctive normal form (CNF) first. Then, it goes over each disjunctive clause and converts it to a functional constraint. While this process is easier to implement, it results in an explosion of functional constraints. A more efficient approach would be to write the propositional formulas in disjunctive normal form (DNF) and then convert them to functional constraints. This process would require including auxiliary variables, as specified in [IMMS Modeling Guide - Integer Programming Tricks](https://download.aimms.com/aimms/download/manuals/AIMMS3OM_IntegerProgrammingTricks.pdf).

#### Class Initialization
##### `UCMPlanner.__init__(self, reclib: Sequence[dict] | str) -> None`
Initializes the `UCMPlanner` with the provided recommendation library (reclib).

##### Parameters
- `reclib` (Sequence[dict] | str): The recommendation library, which can be either a sequence of dictionaries representing courses or a string path to a JSON file containing this data.

#### Internal Workings
- Upon initialization, the planner processes course data, normalizes period information, and establishes various mappings and constraints crucial for the planning algorithm.
- The planner formulates the academic trajectory as a constrained scheduling problem, aiming to maximize the total preference score while adhering to numerous constraints, including EER and course prerequisites.
- Course prerequisites, initially in natural language, are converted into propositional logic and subsequently into functional constraints for the ILP solver.

#### Methods
##### `_update_ranks(self, courses: Sequence[dict])`
Updates course rankings based on provided scores.

##### Parameters
- `courses` (Sequence[dict]): A sequence of dictionaries, each containing a course code and a score.

##### `plan(self, courses: Sequence[dict])`
Generates an academic trajectory based on the updated course scores.

##### Parameters
- `courses` (Sequence[dict]): A sequence of dictionaries containing course information and up-to-date scores.

##### Returns
- A structured academic plan (`pretty_tree`), organizing courses into periods and years, or `None` if no feasible plan is found.

**Notes**
- The planning process involves the generation of linear constraints and the application of a Mixed Integer Linear Programming (MILP) solver from `scipy`.
- Additional constraints related to general education and concentration requirements (`gedu_ineq` and `conc_ineq`) are incorporated into the planning process as needed.
- Typically, an entire academic trajectory can be generated in under four seconds. But sometimes the process may take longer.
